In [2]:
# Colab cell — run from a fresh runtime (recommended)
# Uninstall possibly-busted installs first (safe to run)
!pip uninstall -y transformers tokenizers tortoise-tts

# Install compatible transformers + tokenizers
!pip install transformers==4.43.4 tokenizers==0.19.1

# (Optional) reinstall torch/torchaudio if needed — match to your CUDA runtime (Colab GPU)
!pip install torch --index-url https://download.pytorch.org/whl/cu121
!pip install torchaudio --index-url https://download.pytorch.org/whl/cu121

# Reinstall tortoise-tts from your repo
%cd /content
!git clone https://github.com/neonbjb/tortoise-tts.git
%cd tortoise-tts
!pip install -r requirements.txt
!python3 setup.py install


Found existing installation: transformers 4.43.4
Uninstalling transformers-4.43.4:
  Successfully uninstalled transformers-4.43.4
Found existing installation: tokenizers 0.19.1
Uninstalling tokenizers-0.19.1:
  Successfully uninstalled tokenizers-0.19.1
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/huggingface_hub-0.16.4-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/progressbar-2.5-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/scipy-1.13.1-py3.12-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A 

In [3]:
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

tts = TextToSpeech()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.o

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

...

autoregressive.pth:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

diffusion_decoder.pth:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

clvp2.pth:   0%|          | 0.00/976M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


vocoder.pth:   0%|          | 0.00/391M [00:00<?, ?B/s]

In [4]:
# This is the text that will be spoken.
text = "hello fellas what are you doing, You know I'm the smartest man in the multiuniverses"

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "high_quality"

In [ ]:
CUSTOM_VOICE_NAME = "martin"

import os
from google.colab import files

custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder, exist_ok=True)
for i, file_data in enumerate(files.upload().values()):
  with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
    f.write(file_data)

Saving IMG_7078.wav to IMG_7078.wav
Saving IMG_7081.wav to IMG_7081.wav
Saving IMG_7082.wav to IMG_7082.wav
Saving IMG_7083.wav to IMG_7083.wav
Saving IMG_7084.wav to IMG_7084.wav
Saving IMG_7085.wav to IMG_7085.wav
Saving IMG_7086.wav to IMG_7086.wav
Saving IMG_7087.wav to IMG_7087.wav


In [ ]:
# Generate speech with the custotm voice.
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')

Generating autoregressive samples..


100%|██████████| 16/16 [04:36<00:00, 17.29s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 16/16 [00:38<00:00,  2.42s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 400/400 [00:29<00:00, 13.40it/s]
